### Download and unzip the English texts collection, which contains over 100,000 documents. The documents will be saved in the /content/enwiki directory

In [ ]:
!gdown https://drive.google.com/uc?id=1wzj7AXu_UmwrMc5Tl1_acMKbYdKWaXMC
!unzip /content/enwiki.zip

# Section A

Simple code for searching phrases

In [ ]:
# This code block prepares documents for further searching

from typing import List, Dict
from tqdm import tqdm
import os

def read_text_fragments(file_path: str) -> List[str]:
    with open(file_path, "r") as f:
        fragments = f.readlines()
    # Remove leading and trailing spaces and new line characters before returning text fragments
    return [fragment.strip() for fragment in fragments if fragment.strip()]

def extract_and_combine_text_fragments(folder_path: str) -> Dict[str, List[str]]:
    """
    This function iterates over all files in the folder and saves their fragments in a dictionary,
    Returns:
        A dictionary, where the dictionary key is the file name, and its value is a list of all fragments in that file.
    Example:
        texts_fragments_dict = {
              "file_1.txt": ["this is an example of some fragment", "this is an example of another fragment"],
              "file_2.txt": ["this is another example"],
              ...
        }
    """

    texts_fragments_dict = {}
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Extract text file fragments
        file_fragments = read_text_fragments(file_path)
        # Add file name and its value to texts_fragments_dict
        texts_fragments_dict[file_name] = file_fragments
    return texts_fragments_dict

# Extract all documents fragments from the specified folder
folder_path = "/content/enwiki"
texts_fragments_dict = extract_and_combine_text_fragments(folder_path=folder_path)

In [ ]:
# Function that searches similar phrases
def search_similar_phrases(phrase: str, texts_fragments: Dict[str, List[str]]) -> List[List[str]]:
    """
    For the given phrase, search for similar phrases in the texts_fragments dict.
    Returns:
        A list containing information about in which file the phrase is written and the corresponding fragment.
        If nothing is found, it returns an empty list.
    Example:
        search_results = [
            ["file_1.txt", "this is an example of some fragment"],
            ["file_1.txt", "this is an example of another fragment"],
            ["file_2.txt", "this is another example"]
            ...
        ]
    """
    search_results = []
    # Iterate over all files
    for file_name in texts_fragments:
        # Iterate over file fragments
        for fragment in texts_fragments[file_name]:
            # Check if the phrase is in our fragment, then save it in the search_results list
            if phrase in fragment:
                search_results.append([file_name, fragment])

    return search_results

In [ ]:
# Example of searching
phrase = "Berisha is elected chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[['17919119.txt', 'Berisha is elected chairman of the Democratic Party.']]


This algorithm works slowly; if we have, for example, a million texts, it can take minutes to return an answer.

Besides this, the proposed method only finds identical texts.

In [ ]:
# 1. If we delete some of the middle words, we will receive an empty result
phrase = "Berisha elected chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[]


In [ ]:
# 2. Or if we delete some word ending
phrase = "Berisha is elect chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[]


In [ ]:
# 3. Or if we make words lowercase
phrase = "berisha is elected chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[]


In [ ]:
# 4. Or if we change some similar meaning words, "elected" -> "appointed"
phrase = "Berisha is appointed chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[]


# Section B

Research and implement a search algorithm that overcomes these limitations and performs efficiently and rapidly on a vast collection of documents. Additionally, you need to explore other potential problems that might occur during the search process and propose solutions to avoid them.

Unfortunately, there is an issue with the RAM limitation in google colab. So it's recomended to run the code for a part of documents. Here is the 10% (10.000 files) of all the documents.

In [ ]:
!gdown https://drive.google.com/uc?id=1_2sN1ZBL2sxvZn46ebBDvoLRL4AenquB&export=download
!unzip /content/enwiki.zip

Here is the text files preprocessing part.

Note: You can uncomment the line "line_list = perform_stemming(line_list)" in the function "normalize_line_and_split", so you will get results with modified word endings. However, preprocessing will consume significantly more time.

In [ ]:
from typing import List, Dict, Tuple, Set
from collections import defaultdict  # Provides a default value for keys that don't exist in the dictionary
import linecache                     # To get a line from a text document
import os                            # Read documents
import re                            # Regular expression to simplify some functions
import time                          # To measure searching time
import nltk                          # ...
from nltk.stem import PorterStemmer  # To cut word endings


def perform_stemming(word_list: List[str]) -> List[str]:
    """
    This function iterates over all tokens in a list, performs stemming on each.
    Returns:
        A list of strings, where each token is stemmed.
    Example:
        stemmed_words = ["run", "write", "histor", ...]
    """
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in word_list]
    return stemmed_words


# Function that splitting a line into a list of tokens
def split_line(input_string: str) -> List[str]:
    return re.split(r'\s+', input_string)


# Function that removes punctuation from a line
def remove_punctuation(input_string: str) -> str:
    return re.sub(r'[^\w\s]', '', input_string)


def normalize_line_and_split(input_line: str) -> List[str]:
    """
    This function tokenizes the given line.
    Note: I could write a function, that does everything here manually(except stemming). That shall decrease preprocessing time.
    Returns:
        A list of strings, where each token is stripped, lowercased, punctuation removed and stemmed(optional).
    Example:
        input_line = " (objects that are proved to exist, but "
        line_list = ["objects", "that", "are", "proven", "to", "exist", "but"] # Not stemmed
        line_list = ["object", "that", "are", "prov", "to", "exist", "but"]   # Stemmed
    """
    input_line = input_line.strip()
    input_line = remove_punctuation(input_line)
    input_line = input_line.lower()
    line_list = split_line(input_line)
    #line_list = perform_stemming(line_list)
    return line_list


# Reads file content via given path and Stores its content to a list of strings, where every string is a line from the file
def read_given_file(file_path: str) -> List[str]:
    with open(file_path, "r", encoding="utf-8") as f:
        return f.readlines()


def append_file_content_to_dictionary(lines: List[str], file_name: str):
    """
    This function iterates over file content, tokenizes it, and appends tokens to a data structure, holding all tokens and their position.
    In token_dictionary key is a token(string) and value is a set of tuples, where elements of tuple are file name(string) and line id(int).
    Example:
        token_dictionary = {
              "example": [("file_name_1.txt", 14), ("file_name_1.txt", 23), ("file_name_2.txt", 57), ...],
              "another": [("file_name_1.txt", 63), ("file_name_3.txt", 35), ...],
              ...
        }
    """
    for line_id in range(len(lines)):
        if lines[line_id].strip():
            token_list = normalize_line_and_split(lines[line_id])
            for token_id in range(len(token_list)):
                token_dictionary[token_list[token_id]].add((file_name, line_id + 1))


# This function iterates over all files, reads their content, tokenizes them and saves their tokens in a dictionary,
def extract_and_combine_text_files():
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        lines = read_given_file(file_path)
        append_file_content_to_dictionary(lines, file_name)


token_dictionary = defaultdict(set)
folder_path = "/content/enwiki"

# Download the "punkt" dataset from the NLTK library for stemming.
nltk.download("punkt")

print("\nPreprocessing files...")
extract_and_combine_text_files()
print("Ended!")

Here are the searching logic functions.

In [ ]:
def get_search_results_intersection(phrase_tokens: List[str]) -> Set[Tuple[str, int]]:
    """
    Finds lines, where all the tokens from a searching phrase appear.
    Returns:
        A set of tuples, where tuples are values from the token_dictionary (file name(string) and line id(int)).
    Example:
        intersection_result = {(file_name_1.txt", 25), (file_name_3.txt", 72), ...}
    """
    intersection_result = Set[Tuple[str, int]]
    # Iterate over all phrase tokens
    for token_id in range(len(phrase_tokens)):
        # If it's first token, initialize result by it's elements
        if token_id == 0:
            intersection_result = token_dictionary[phrase_tokens[token_id]]
        # Intersect result with token_id-th token's value
        else:
            intersection_result = intersection_result.intersection(token_dictionary[phrase_tokens[token_id]])
    return intersection_result


# Function that receives file name and line number and returns its content
def get_line_content(file_name: str, line_number: int) -> str:
    file_path = os.path.join(folder_path, file_name)
    return linecache.getline(file_path, line_number)[:-1]


def get_matches_from_line(line_tokens_list: List[str], phrase_tokens: List[str]) -> List[int]:
    """
    Function, that find given phrase on the given line.
    Note: There is a version of this function below, that enables results with a middle word missing from the searched phrase
    Returns:
        A list of positions (token_id (int)), where phrase appears on the line.
    Example:
        result = [10, 33, ...]
    """
    result = []
    # Iterate over all tokens in the given line
    for token_id in range(len(line_tokens_list)):
        # Checks if the token from line matches with the first token of the searching phrase
        if line_tokens_list[token_id] != phrase_tokens[0]:
            continue
        # This logic block checks if the next tokens of the line match with the tokens of the phrase
        exact_match = True
        # Iterate over phrase tokens and check if the corresponding token of the line matches with it
        for token_from_phrase_id in range(1, len(phrase_tokens)):
            # This condition prevents index going out of bounds
            if token_id + token_from_phrase_id >= len(line_tokens_list):
                exact_match = False
                break
            if phrase_tokens[token_from_phrase_id] != line_tokens_list[token_id + token_from_phrase_id]:
                exact_match = False
                break
        # If none of conditions above happen, then it's a match
        if exact_match:
            # Append token position from the line, to the list
            result.append(token_id)
    return result


# Global variable to measure time spent on a search request
time_taken = 0.0


def search_similar_phrases(phrase: str) -> List[tuple[str, int, int, str]]:
    """
    Function that combines functions above to find the phrase positions in text documents
    Returns:
        A list of tuples, where elements of it are file name, line number, position on the line, and line content.
    Example:
        final_result = [
              ("file_name_1.txt", 14, 142, "Line content from file_name_1.txt and line 14"),
              ("file_name_3.txt", 66, 20, "Line content from file_name_3.txt and line 66"),
              ...
        ]
    """
    global time_taken
    # Start measuring time
    start_time = time.time()

    # Tokenize phrase
    phrase_tokens = normalize_line_and_split(phrase)
    final_result = []

    # Intersects all phrase tokens positions, to determine lines, where phrase tokens appeare together
    intersection_result = get_search_results_intersection(phrase_tokens)

    # This code block appends to the final_result exact position where phrase appears in text documents
    for match in intersection_result:
        # Get file name and line number, where all tokens from phrase appear together
        file_name = match[0]
        line_number = int(match[1])
        # Get line content, tokenize, then get positions (if there are any), where phrase tokens appear consequently
        line_content = get_line_content(file_name, line_number)
        line_tokens_list = normalize_line_and_split(line_content)
        match_list = get_matches_from_line(line_tokens_list, phrase_tokens)
        # Append that positions to the final result
        for match_id in range(len(match_list)):
            token_number = match_list[match_id] + 1
            final_result.append((file_name, line_number, token_number, line_content))

    # Stop the timer
    end_time = time.time()
    time_taken = end_time - start_time

    return final_result


# A function to print results found
def print_results(final_result: List[tuple[str, int, int, str]]):
    print()
    formatted_time = "{:.5f}".format(time_taken)
    # Case, when nothing found
    if len(final_result) == 0:
        print("Not found! " + formatted_time + " seconds spent.")
        print()
        return
    # Iterate over results and print it
    for result in final_result:
        file_name = result[0]
        line_number = result[1]
        word_number = result[2]
        line_content = result[3]
        print(file_name + ", line " + str(line_number) + ", word " + str(word_number) + ": " + line_content)
    print()
    print(f"Found " + str(len(final_result)) + " results in " + formatted_time + " seconds.")
    print()

In [ ]:
def print_results(final_result: List[tuple[str, int, int, str]]):
    print()
    formatted_time = "{:.5f}".format(time_taken)
    print(f"Found " + str(len(final_result)) + " results in " + formatted_time + " seconds.")
    print()

Here you can search for a specific phrase.

Note: If there are many lines of result, and google colab is not showing it properly, to get searching time and results count, you can use function above.

In [ ]:
user_input = input("Enter phrase to search: ")

results = search_similar_phrases(user_input)
print_results(results)

You can run this code fragment to enable search results, including deleted middle words from the phrase being searched.

In [ ]:
def get_matches_from_line(line_tokens_list: List[str], phrase_tokens: List[str]) -> List[int]:
    result = []
    for token_number in range(len(line_tokens_list)):
        if line_tokens_list[token_number] != phrase_tokens[0]:
            continue
        exact_match = True
        # Everything above here is not changed
        # Bool variable to ignore 1 middle word missing from phrase searched on the line
        middle_word_missing = False
        phrase_token_index = 1
        # Iterate over line tokens and check if the corresponding token of the phrase matches with it
        for current_token_number in range(token_number + 1, token_number + len(phrase_tokens)):
            # Case, where phrase token and line tokens are not the corresponding
            if phrase_tokens[phrase_token_index] != line_tokens_list[current_token_number]:
                # Case, where there is already one missing token in phrase
                if middle_word_missing:
                    exact_match = False
                    break
                # If it's first time, then don't change phrase token and check for the next line token
                else:
                    phrase_token_index -= 1
                    middle_word_missing = True
            phrase_token_index += 1
        # Again, no changes here
        if exact_match:
            result.append(token_number)
    return result

Here is an example of enabled/disabled missing middle word to try.

In [ ]:
results = search_similar_phrases("media outlets suggested displaying")
print_results(results)

## **Advantages:**

In [ ]:
# 1. If we search for a concrete phrase, results will be found very quickly, which is a realistic case of using this algorithm
#    missing middle word - disabled, stemming - disabled

results = search_similar_phrases("provide nonconstructive proofs")
print_results(results)


840.txt, line 52, word 13: As discussed above, in ZFC, the axiom of choice is able to provide "nonconstructive proofs" in which the existence of an object is proved although no explicit example is constructed. ZFC, however, is still formalized in classical logic. The axiom of choice has also been thoroughly studied in the context of constructive mathematics, where non-classical logic is employed. The status of the axiom of choice varies between different varieties of constructive mathematics.

Found 1 results in 0.00012 seconds.



In [ ]:
# 2. If we search without punctuation, using uppercase/lowercase characters, which are not the same as in text, there is no difference: algorithm will find it
#    missing middle word - disabled, stemming - disabled

results = search_similar_phrases("watercress nasturtium officinale is")
print_results(results)


15374.txt, line 52, word 1: Watercress ("Nasturtium Officinale") is a rapidly growing aquatic or semi aquatic perennial plant. It contains health promoting phytochemicals endowed in therapeutic properties. Because chemical agents do not provide efficient microbial reductions, watercress has been tested with gamma irradiation treatment in order to improve both safety and the shelf life of the product. It is traditionally used on horticultural products to prevent sprouting and post-packaging contamination, delay post-harvest ripening, maturation and senescence.

Found 1 results in 0.00032 seconds.



In [ ]:
# 2.1. If we search a phrase with an unstripped content, or search for a number, when format is not corresponding original, that's also shall be found
#    missing middle word - disabled, stemming - disabled

results = search_similar_phrases("  451.000        students   ")
print_results(results)


19226.txt, line 106, word 7: There are almost half a million (451,000) students enrolled in electronics engineering programs with an additional 114,000 electronics engineers entering the Mexican workforce each year and Mexico had over half a million (580,000) certified electronic engineering professionals employed in 2007. From the late 1990s, the Mexican electronics industry began to shift away from simple line assembly to more advanced work such as research, design, and the manufacture of advanced electronics systems such as LCD panels, semiconductors, printed circuit boards, microelectronics, microprocessors, chipsets and heavy electronic industrial equipment and in 2006 the number of certified engineers being graduated annually in Mexico surpassed that of the United States. Many Korean, Japanese and American appliances sold in the US are actually of Mexican design and origin but sold under the OEM's client names. In 2008 one out of every four consumer appliances sold in the United

In [ ]:
# 3. It's possible to find results, even if there is a 1 missing word in a phrase
#    missing middle word - enabled, stemming - disabled

results = search_similar_phrases("gradually become hospitable")
print_results(results)


9813.txt, line 45, word 10: It has also been suggested that the oceans have gradually become more hospitable to life over the last 500 million years, and thus less vulnerable to mass extinctions, but susceptibility to extinction at a taxonomic level does not appear to make mass extinctions more or less probable.

Found 1 results in 0.00065 seconds.



In [ ]:
# 4. It's possible to find results, where word endings are different
#    missing middle word - disabled, stemming - enabled

results = search_similar_phrases("period of increased geomagnetic reversal")
print_results(results)


9813.txt, line 109, word 5: One theory is that periods of increased geomagnetic reversals will weaken Earth's magnetic field long enough to expose the atmosphere to the solar winds, causing oxygen ions to escape the atmosphere in a rate increased by 3–4 orders, resulting in a disastrous decrease in oxygen.

Found 1 results in 0.01931 seconds.



# **Disadvantages:**

In [ ]:
# 1. There are cases, where punctuation can interfere searching results
#    missing middle word - disabled, stemming - disabled

results = search_similar_phrases("100,000-year cycle of radiation")
print_results(results)


15361.txt, line 92, word 8: Kuhle explains the interglacial periods by the 100,000-year cycle of radiation changes due to variations in Earth's orbit. This comparatively insignificant warming, when combined with the lowering of the Nordic inland ice areas and Tibet due to the weight of the superimposed ice-load, has led to the repeated complete thawing of the inland ice areas.

Found 1 results in 0.00015 seconds.



In [ ]:
results = search_similar_phrases("100,000 year cycle of radiation")
print_results(results)


Not found! 0.00050 seconds spent.



In [ ]:
# 1.1. It's also possible to find not exactly what we wanted due to punctuation removal
#    missing middle word - disabled, stemming - disabled

results = search_similar_phrases("10.74 km/h")
print_results(results)


56824.txt, line 66, word 9: The maximum wind speed record in Accra is 107.4 km/h (58 knots). Strong winds associated with thunderstorm activity often cause damage to property by removing roofing material. Several areas of Accra experience microclimatic effects. Low-profile drainage basins with a north-south orientation are not as well ventilated as those oriented east-west.

Found 1 results in 0.00012 seconds.



In [ ]:
# 2. Searching for example for stop-words can lead to very poor performance due to a very big set of intersection of that stop-words in the same line
#    missing middle word - disabled, stemming - disabled

results = search_similar_phrases("at the a is")
print_results(results)


Not found! 5.71506 seconds spent.



In [ ]:
# 3. If using synonyms in the searching phrase, result cannot be found
#    missing middle word - disabled, stemming - disabled

results = search_similar_phrases("direct connection between these can be proven")
print_results(results)


15236.txt, line 52, word 14: Following the 2013 NSA spying scandal, ICANN endorsed the Montevideo Statement, although no direct connection between these can be proven.

Found 1 results in 0.00046 seconds.



In [ ]:
results = search_similar_phrases("direct association between these can be proven")
print_results(results)


Not found! 0.00079 seconds spent.

